Steve Markstrom
Thu Jan  9 09:05:21 MST 2020 

This notebook contains simple examples to show how to use notebooks with xarray on ONMH output ncf files. These examples use the "onhm" python package that is availaible from https://github.com/nhm-usgs/pangeo/src/onhm

The next cell demonstrates the get_DataSet() function from the onhm package. This function lazy loads all of the netcdf files found in the specified path into xarray DataArrays and loads each of these DataArrays into a single xarray DataSet. 

In [1]:
    # path is the path to were there are output PRMS ncf files
    # ext is the file extention on these files
    # The python package "onhm" can be built from the source code at https://github.com/nhm-usgs/pangeo/src/onhm
    
    import onhm
    
    # For example, to see the results of the historical ONHM runs on Denali, the path would be
    path = '/caldera/projects/usgs/water/wbeep/onhm/historical/output/'
    #path = '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/output/'
    ext = '.nc'
    
    ds_out = onhm.reader.get_DataSet(path, ext)
    print(ds_out)

<xarray.Dataset>
Dimensions:              (hruid: 109951, segid: 56460, time: 14569)
Coordinates:
  * hruid                (hruid) int32 1 2 3 4 5 ... 109948 109949 109950 109951
  * time                 (time) datetime64[ns] 1979-10-01T05:00:00 ... 2019-08-20T05:00:00
  * segid                (segid) int32 1 2 3 4 5 ... 56457 56458 56459 56460
Data variables:
    hru_lat              (hruid) float64 ...
    hru_lon              (hruid) float64 ...
    unused_potet         (time, hruid) float64 ...
    gwres_in             (time, hruid) float64 ...
    newsnow_out          object None
    seg_lat              (segid) float64 ...
    seg_lon              (segid) float64 ...
    segment_delta_flow   (time, segid) float64 ...
    transp_on            (time, hruid) float64 ...
    tmaxf                (time, hruid) float64 ...
    dprst_vol_open_frac  (time, hruid) float64 ...
    potet                (time, hruid) float64 ...
    hru_streamflow_out   (time, hruid) float64 ...
    pref_flo

The latitude and longitudes of each of the modeling features (HRUs and Stream Segments) are recorded in the onhm output ncf files. The next cell shows how to get the geographic coordinates (decimal degrees) of any HRU or Stream Segment. These examples assume that you know the ID of the HRU and the Stream Segment.

In [2]:
    # Get the coordinates of the centroid of an HRU
    hru_id = 50471
    lat, lon = onhm.reader.get_feat_coord(feat='hru', data_set=ds_out, feat_id=hru_id)
    print("Coordinates of HRU " + str(hru_id) + " = " + str(lat) + "," + str(lon))
    
    # Get the coordinates of the midpoint of a stream segment
    seg_id = 100
    lat, lon = onhm.reader.get_feat_coord(feat='seg', data_set=ds_out, feat_id=seg_id)
    print("Coordinates of stream segment " + str(seg_id) + " = " + str(lat) + "," + str(lon ))    
    

Coordinates of HRU 50471 = 40.013438,-100.1355
Coordinates of stream segment 100 = 46.094257776,-67.9908220109


To get the IDs of the HRUs within a lat/lon bounding box:

In [3]:
    # Get the HRU IDs by bounding lat/lon box
    hru_ids = onhm.reader.get_hrus_for_box(ds_out, lat_min=40.0, lat_max=41.0,
                                               lon_min=-101.0, lon_max=-100.0)
    print('hru_ids=', hru_ids)

hru_ids= [50471 50474 50482 50495 50506 50507 50514 50515 50517 50524 50527 50528
 50529 50530 50534 50538 50539 50540 50541 50542 50543 50546 50547 50550
 50552 50553 50554 50555 50559 50560 50561 50562 50566 50568 50573 50574
 50578 50587 50591 50595 50596 50598 50600 50601 50603 50604 50606 50608
 50610 50612 50614 50615 50617 50619 50620 50623 50626 50627 50630 50635
 50638 50639 50640 50641 50644 50645 50647 50649 50651 50652 50653 50654
 50655 50657 50661 50662 50666 50668 50669 50670 50671 50672 50673 50674
 50678 50679 50681 50685 50686 50694 50696 50697 50700 50702 50704 50705
 50710 51294 51295 51296 51299 51302 51387 51388 51392 51393 51394 51591
 51592 51593 51594 51601 51602 51603 51604 51605 51606 52192 55476 55498
 55502]


To get the IDs of the Stream Segments within a lat/lon bounding box:

In [4]:
    # Get the Stream Segment IDs by bounding lat/lon box
    seg_ids = onhm.reader.get_segs_for_box(ds_out, lat_min=40.0, lat_max=41.0,
                                               lon_min=-101.0, lon_max=-100.0)
    print('seg_ids=', seg_ids)

seg_ids= [26236 26237 26238 28096 28097 28098 28099 28101 28102 28103 28104 28105
 28106 28110 28140 28141 28142 28143 28144 28145 28146 28147 28148 28149
 28150 28151 28152 28153 28154 28155 28156 28157 28158 28159 28164 28165
 28166 28167 28185 28200 28201 28202 28203 28204 28205 28206 28207 28209
 28210 28211 28212 28213 28214 28215 28216 28217 28218 28219 28220 28221
 28222 28421]


To get the HRU values for an output variable for a single time step

In [5]:
    # Get the values for all HRUs for an output variable for a single time step
    var_name = 'hru_rain'
    time_step = '1979-10-02'
    sel1 = ds_out[var_name].sel(time=time_step)
    
    # This is an xarray DataArray
    print(sel1)
    # This is a numpy ndarray
    print("numpy.ndarray=", sel1.values)
    print('number of hrus = ', sel1.values.size)

<xarray.DataArray 'hru_rain' (time: 1, hruid: 109951)>
[109951 values with dtype=float64]
Coordinates:
  * hruid    (hruid) int32 1 2 3 4 5 6 ... 109947 109948 109949 109950 109951
  * time     (time) datetime64[ns] 1979-10-02T05:00:00
Attributes:
    long_name:      Rain distributed to each HRU
    standard_name:  hru_rain
    units:          mm
numpy.ndarray= [[0.15 3.81 3.81 ... 0.   0.   0.  ]]
number of hrus =  109951


To get the time series for a paticular variable for a single HRU

In [6]:
    # Get the time series for a paticular variable for a single HRU.
    var_name = 'hru_rain'
    hru_id = 50471
    sel1 = ds_out[var_name].sel(hruid=hru_id)
    
    # This is an xarray DataArray
    print(sel1)
    # This is a numpy ndarray
    print(sel1.values)
    print('number of time steps = ', sel1.values.size)

<xarray.DataArray 'hru_rain' (time: 14569)>
array([0.   , 0.   , 0.   , ..., 0.   , 1.27 , 0.508])
Coordinates:
    hruid    int32 50471
  * time     (time) datetime64[ns] 1979-10-01T05:00:00 ... 2019-08-20T05:00:00
Attributes:
    long_name:      Rain distributed to each HRU
    standard_name:  hru_rain
    units:          mm
[0.    0.    0.    ... 0.    1.27  0.508]
number of time steps =  14569
